## Using pyspark to create a linear regression model to predict the number of crew members a cruise line will need. 

By: Matt Purvis

In [0]:
# Import SparkSession
from pyspark.sql import SparkSession

In [0]:
# Create SparkSession
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [0]:
# Import LinearRegression
from pyspark.ml.regression import LinearRegression

In [0]:
# Read in the data

df = spark.read.csv('dbfs:/FileStore/tables/cruise_ship_info.csv', inferSchema = True, header = True)

In [0]:
# Preview the data
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [0]:
# Import StringIndexer to create a dummy variable for Cruise_Line
from pyspark.ml.feature import StringIndexer

In [0]:
# Create indexer object
indexer = StringIndexer(inputCol="Cruise_line", outputCol="CruiseIndex")

In [0]:
# Perform the dummy variable encoding
indexed = indexer.fit(df).transform(df)

In [0]:
# Show the resulting df
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|CruiseIndex|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------+
only showing top 20 rows

In [0]:
# Import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
# Show column names from encoded df
indexed.columns

Out[76]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'CruiseIndex']

In [0]:
# Create an assembler object to transform the features into a features vector
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','CruiseIndex'], outputCol = 'features') 

In [0]:
# Perform the transformation
output = assembler.transform(indexed)

In [0]:
# Grab only the new features column and the labels
final_data = output.select(['features', 'crew'])

In [0]:
# Display the final_data df
final_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
# Split into train and test sets
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [0]:
# Describe the training set
train_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 108|
 mean| 7.772314814814824|
 stddev|3.6129418480182682|
 min| 0.59|
 max| 21.0|
+-------+------------------+

In [0]:
# Describe the test set
test_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 50|
 mean| 7.841400000000001|
 stddev|3.2894767340222164|
 min| 1.46|
 max| 13.6|
+-------+------------------+

In [0]:
# Create linear regression model and indicate label column
lr = LinearRegression(labelCol = 'crew')

In [0]:
# Fit the model to the training data
lr_model = lr.fit(train_data)

In [0]:
# Evaluate the model on the test set
test_results = lr_model.evaluate(test_data)

In [0]:
# Get the rmse 
test_results.rootMeanSquaredError

Out[87]: 1.0338201619909304

In [0]:
# Compare rmse to mean and std dev - indicates that our rmse is not bad!
df.select('crew').describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 158|
 mean|7.794177215189873|
 stddev|3.503486564627034|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [0]:
# Display the explained variance for the model - not bad at all!
test_results.r2

Out[88]: 0.8992116624902803

In [0]:
# Demonstrate how we can use the model to predict unkown crew member numbers for unlabeled data
unlabeled_data = test_data.select('features')

In [0]:
# Demonstrate how we can use the model to predict unkown crew member numbers for unlabeled data
predictions = lr_model.transform(unlabeled_data)

In [0]:
# Demonstrate how we can use the model to predict unkown crew member numbers for unlabeled data
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[5.0,86.0,21.04,9...| 9.211724270216635|
[5.0,160.0,36.34,...|15.447747399523058|
[6.0,90.0,20.0,9....|10.050561163966563|
[6.0,110.23899999...| 11.026798212748|
[7.0,89.6,25.5,9....|11.008224710922237|
[7.0,116.0,31.0,9...|12.773733648258213|
[8.0,77.499,19.5,...| 8.568693594251702|
[8.0,91.0,22.44,9...|10.074915385784365|
[9.0,81.0,21.44,9...| 9.465722689995227|
[9.0,88.5,21.24,9...| 9.543317928869588|
[9.0,90.09,25.01,...| 9.188407126023643|
[9.0,116.0,26.0,9...|11.368887735216582|
[10.0,58.825,15.6...| 7.167175353175648|
[10.0,68.0,10.8,7...| 6.606377628805447|
[10.0,86.0,21.14,...| 9.645543403732704|
[10.0,105.0,27.2,...|11.386867068406866|
[11.0,85.0,18.48,...| 8.782191505631303|
[11.0,90.0,22.4,9...| 10.03266166704736|
[11.0,108.977,26....|11.228533530220698|
[11.0,138.0,31.14...|13.296925208771793|
+--------------------+------------------+
only showing top 20 rows

In [0]:
# Look at correlations to determine if our model results are realistic or not
from pyspark.sql.functions import corr

In [0]:
# Correlation for passengers is really high
df.select(corr('crew','passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [0]:
# Correlation for cabins is really high
df.select(corr('crew','cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+

In [0]:
# Our label column is highly correlated to features. Therefore we can be confident that our model is realistic and is a good model. 